In [12]:
from llama_index.core.tools import FunctionTool
from llama_index.retrievers.bm25 import BM25Retriever
import datasets
from llama_index.core.schema import Document
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.openai import OpenAI

In [11]:
import os
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file

open_api_key = os.getenv('OPENAI_API_KEY')

In [7]:
# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        text="\n".join([
            f"Name: {guest_dataset['name'][i]}",
            f"Relation: {guest_dataset['relation'][i]}",
            f"Description: {guest_dataset['description'][i]}",
            f"Email: {guest_dataset['email'][i]}"
        ]),
        metadata={"name": guest_dataset['name'][i]}
    )
    for i in range(len(guest_dataset))
]

In [8]:

bm25_retriever = BM25Retriever.from_defaults(nodes=docs)

In [9]:
def get_guest_info_retriever(query: str) -> str:
    """Retrieves detailed information about gala guests based on their name or relation."""
    results = bm25_retriever.retrieve(query)
    if results:
        return "\n\n".join([doc.text for doc in results[:3]])
    else:
        return "No matching guest information found."

# Initialize the tool
guest_info_tool = FunctionTool.from_defaults(get_guest_info_retriever)

In [14]:
# Initialize the model with the required parameters
openai_model_name =  "gpt-4o-mini"
llm = OpenAI(model=openai_model_name, api_key=open_api_key)

In [15]:
# Create Alfred, our gala agent, with the guest info tool
alfred = AgentWorkflow.from_tools_or_functions(
    [guest_info_tool],
    llm=llm,
)

# Example query Alfred might receive during the gala
response = await alfred.run("Tell me about our guest named 'Lady Ada Lovelace'.")

print("🎩 Alfred's Response:")
print(response)

🎩 Alfred's Response:
Lady Ada Lovelace is your best friend and an esteemed mathematician. She is renowned for her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her contributions to Charles Babbage's Analytical Engine. 

Here are her details:
- **Name:** Ada Lovelace
- **Relation:** Best friend
- **Description:** Renowned mathematician and the first computer programmer.
- **Email:** ada.lovelace@example.com

If you need more information or have any specific questions about her, feel free to ask!
